In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_627402/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [8]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_52_9_12,0.999809,0.827260,0.998906,0.999495,0.999235,0.000271,0.245180,0.000970,0.000475,0.000722,0.021093,0.016472,1.000007,0.017173,1350.424397,2163.414572,"Hidden Size=[26, 21], regularizer=0.0, learnin..."
1,model_52_9_13,0.999809,0.827238,0.998890,0.999460,0.999209,0.000271,0.245210,0.000985,0.000508,0.000746,0.021083,0.016472,1.000007,0.017174,1350.424297,2163.414472,"Hidden Size=[26, 21], regularizer=0.0, learnin..."
2,model_52_9_14,0.999808,0.827219,0.998874,0.999427,0.999186,0.000272,0.245238,0.000998,0.000538,0.000768,0.021072,0.016499,1.000007,0.017201,1350.417818,2163.407993,"Hidden Size=[26, 21], regularizer=0.0, learnin..."
3,model_52_9_11,0.999808,0.827282,0.998924,0.999532,0.999262,0.000273,0.245148,0.000954,0.000440,0.000697,0.021104,0.016508,1.000007,0.017211,1350.415558,2163.405733,"Hidden Size=[26, 21], regularizer=0.0, learnin..."
4,model_52_9_15,0.999807,0.827200,0.998860,0.999397,0.999164,0.000274,0.245264,0.001011,0.000567,0.000789,0.021065,0.016545,1.000007,0.017249,1350.406778,2163.396953,"Hidden Size=[26, 21], regularizer=0.0, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,model_22_6_9,0.999455,0.804657,0.999004,0.998680,0.999098,0.000773,0.277261,0.002197,0.000353,0.001275,0.175026,0.027800,1.000023,0.028984,1220.330838,1955.312960,"Hidden Size=[30, 16], regularizer=0.2, learnin..."
106,model_2_9_6,0.999445,0.784592,0.999069,0.999613,0.999564,0.000788,0.305741,0.000695,0.000833,0.000764,0.036794,0.028074,1.000010,0.029269,2668.291711,4285.739930,"Hidden Size=[35, 33], regularizer=0.2, learnin..."
112,model_22_6_8,0.999429,0.804630,0.999119,0.998842,0.999204,0.000811,0.277299,0.001942,0.000309,0.001126,0.189891,0.028473,1.000024,0.029685,1220.235227,1955.217350,"Hidden Size=[30, 16], regularizer=0.2, learnin..."
124,model_22_6_7,0.999389,0.804593,0.999239,0.999009,0.999313,0.000867,0.277352,0.001678,0.000265,0.000971,0.206435,0.029443,1.000025,0.030696,1220.101254,1955.083376,"Hidden Size=[30, 16], regularizer=0.2, learnin..."
